## 使用SQL语句汇总

In [2]:
library(sqldf)


Loading required package: gsubfn

Loading required package: proto

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: RSQLite



In [4]:
sale <- read.csv("./data/sale.csv")
sqldf("select year,sum(sale)as sum_sale,sum(profit) as sum_profit from sale group by year")


year,sum_sale,sum_profit
<int>,<int>,<int>
2010,132834,10587
2011,126953,10916
2012,127909,10785


In [5]:
# 汇总某列；定位低于全国销售平均水平地域
sqldf("select distinct market from sale")


market
<chr>
¶«
ÄÏ
Î÷
±±


In [7]:
sqldf("select avg(sale)as all_avg_sale from sale")


all_avg_sale
<dbl>
32308


In [9]:
sqldf("select market,avg(sale)as mavg_sale from sale group by market having mavg_sale<32308 order by mavg_sale")


market,mavg_sale
<chr>,<dbl>
±±,31394.33
ÄÏ,31753.67



- where引导的条件字节分组前处理，放在group by之前
- having引导的条件字节一般跟在group by后面，相当于分组后的再处理.

In [11]:
sqldf("select market,sum(sale)as sum_sale from sale where year>=2011 group by market having sum_sale>63000 order by sum_sale desc")


market,sum_sale
<chr>,<int>
Î÷,66278
¶«,63270
ÄÏ,63015


In [12]:
sqldf("select market,avg(sale)as mavg_sale from sale group by market having mavg_sale<(select avg(sale)as all_avg_sale from sale) order by mavg_sale")


market,mavg_sale
<chr>,<dbl>
±±,31394.33
ÄÏ,31753.67


## 纵向连接表


In [19]:
x <- data.frame(id = c(1, 1, 1, 2, 3, 4, 6), x1 = c("a", "a", "b", "c", "v", "e", "g"))
y <- data.frame(id = c(1, 2, 3, 3, 5), y1 = c("x", "y", "z", "v", "w"))
x
y


id,x1
<dbl>,<chr>
1,a
1,a
1,b
2,c
3,v
4,e
6,g


id,y1
<dbl>,<chr>
1,x
2,y
3,z
3,v
5,w


In [16]:
sqldf("select*from x union select*from y")


id,x1
<dbl>,<chr>
1,a
1,b
1,x
2,c
2,y
3,v
3,z
4,e
5,w


In [17]:
sqldf("select * from   x   INTERSECT   select * from y")


id,x1
<dbl>,<chr>
3,v


 求并集中如果使用union all则相当于rbind,区别在于rbind 要求名字必须相同

In [20]:
sqldf("select * from x EXCEPT select * from y")


id,x1
<dbl>,<chr>
1,a
1,b
2,c
4,e
6,g


## 横向连接表-交叉连接

In [21]:
x <- data.frame(id = c(1, 2, 3), x1 = c("a", "b", "c"))
y <- data.frame(id = c(4, 3), y1 = c("d", "e"))
sqldf("select*from x,y")


id,x1,id,y1
<dbl>,<chr>,<dbl>,<chr>
1,a,4,d
1,a,3,e
2,b,4,d
2,b,3,e
3,c,4,d
3,c,3,e


## 横向连接表-内连接

In [22]:
merge(x, y, by = "id", all = F) ## id


id,x1,y1
<dbl>,<chr>,<chr>
3,c,e


In [23]:
library(dplyr, warn.conflicts = F)
inner_join(x, y, by = "id")


id,x1,y1
<dbl>,<chr>,<chr>
3,c,e


In [24]:
sqldf("select *from x as a inner join y as b on a.id=b.id")


id,x1,id,y1
<dbl>,<chr>,<dbl>,<chr>
3,c,3,e


## 横向连接表-左连接

In [25]:
merge(x, y, by = "id", all.x = TRUE)


id,x1,y1
<dbl>,<chr>,<chr>
1,a,NA
2,b,NA
3,c,e


In [26]:
left_join(x, y, by = "id")


id,x1,y1
<dbl>,<chr>,<chr>
1,a,NA
2,b,NA
3,c,e


In [27]:
sqldf("select *from x as a left join y as b on a.id=b.id")


id,x1,id,y1
<dbl>,<chr>,<dbl>,<chr>
1,a,NA,NA
2,b,NA,NA
3,c,3,e


## 横向连接表-右连接



In [28]:
merge(x, y, by = "id", all.y = TRUE)


id,x1,y1
<dbl>,<chr>,<chr>
3,c,e
4,NA,d


In [29]:
right_join(x, y, by = "id")


id,x1,y1
<dbl>,<chr>,<chr>
3,c,e
4,NA,d


In [30]:
sqldf("select*from x as a right join y as b on a.id=b.id")


id,x1,id,y1
<dbl>,<chr>,<dbl>,<chr>
3,c,3,e
NA,NA,4,d


## 横向连接表-全连接


In [31]:
merge(x, y, by = "id", all = TRUE)
full_join(x, y, by = "id")


id,x1,y1
<dbl>,<chr>,<chr>
1,a,NA
2,b,NA
3,c,e
4,NA,d


id,x1,y1
<dbl>,<chr>,<chr>
1,a,NA
2,b,NA
3,c,e
4,NA,d


$x


$y


$...

